In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
...

driver = webdriver.Chrome('C:/Users/jmlee/Code/chromedriver_win32/chromedriver.exe')
driver.get('https://www.youtube.com/channel/UCE6e9tjKl3l3nGlAcxH7GQg/videos')

#여기서부터
num_of_pagedowns=100
body = driver.find_element_by_xpath('/html/body')

#자동으로 스크롤을 내려서 더이상 영상이 로드 안될때까지 내려줌.
while num_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.3)
    num_of_pagedowns -= 1
    try:
        driver.find_element_by_xpath("""//*[@id="feed-main-what_to_watch"]/button""").click()
    except:
        None
        
page = driver.page_source
page

In [ ]:
from bs4 import BeautifulSoup
#태그, 클래스
soup = BeautifulSoup(page,'lxml')

all_title = soup.find_all('a','yt-simple-endpoint style-scope ytd-grid-video-renderer')
title = [soup.find_all('a','yt-simple-endpoint style-scope ytd-grid-video-renderer')[n].string for n in range(0,len(all_title))]
title

In [ ]:
from bs4 import BeautifulSoup

...


soup = BeautifulSoup(page,'lxml')

#채널명('태그','class명')
chennel = soup.find('yt-formatted-string','style-scope ytd-channel-name').string
#구독자 수
sub_num = soup.find('yt-formatted-string','style-scope ytd-c4-tabbed-header-renderer').string
#조회수, 올린지 얼마나 되었는지(업로드 시점)
c = soup.find_all('span','style-scope ytd-grid-video-renderer')
view_num = [soup.find_all('span','style-scope ytd-grid-video-renderer')[n].string for n in range(0,len(c))]
view_num

In [ ]:
from time import localtime, strftime

...

#현재 시간
extract_date = strftime("%Y/%m/%d %H:%M:%S", localtime())
extract_date

In [ ]:
...

youtube_video_list = []

x = 0		#조회수의 index
y = 1		#업로드 시점의 index

for i in range(0,len(all_title)):
    roww = []
    roww.append(title[i])
#     roww.append(video_time[i].strip())
    roww.append(chennel)
    roww.append(sub_num)
    roww.append(view_num[x])
    x += 2		#조회수만 append
    roww.append(view_num[y])
    y += 2		#업로드 시점만 append
    roww.append(extract_date)
    youtube_video_list.append(roww)			#2차원 list로 만들어줌
    
youtube_video_list

In [ ]:
import csv
#-*- coding: utf-8 -*-
...

csvfile = open("C:\\Users\\jmlee\\Code\\army.txt","w",-1,"utf-8")
csvwriter = csv.writer(csvfile)
for row in youtube_video_list:
    csvwriter.writerow(row)
csvfile.close()

# 후처리

In [4]:
#python3 이용
import os 
import sys
os.environ["SPARK_HOME"]=os.path.join(os.path.expanduser("~"),"Downloads\spark-2.4.2-bin-hadoop2.7")

os.environ["PYLIB"]=os.path.join(os.environ["SPARK_HOME"],'python','lib')
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'py4j-0.10.7-src.zip'))
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'pyspark.zip'))

In [5]:
import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

In [6]:
armyRdd=spark.sparkContext.textFile(os.path.join("crawling txt","army.txt"))

In [7]:
armyList=armyRdd.collect()

In [8]:
print(armyList)

['뜨~끈하고 든든한 국밥 한 그릇 먹고 가세요  Pork and Rice Soup 豚クッパ  mukbang eating show,아미아미,구독자 4.21만명,조회수 1.8만회,2일 전,2019/11/17 22:39:20', '', '해물짬뽕 한 그릇+탕수육 중식먹방!! Korean Chinese food mukbang Jjambbong & Chinese sweet-and-sour pork,아미아미,구독자 4.21만명,조회수 3.7만회,3일 전,2019/11/17 22:39:20', '', '김밥천국에서 소소하게 먹방 Korean food mukbang,아미아미,구독자 4.21만명,조회수 2.9만회,4일 전,2019/11/17 22:39:20', '', '라면 27봉지 먹었습니다... korean mukbang eating show Ramen ラーメン,아미아미,구독자 4.21만명,조회수 13만회,6일 전,2019/11/17 22:39:20', '', '당면 가~득 넣고 뚝배기 불고기 만들어 먹방!! Bulgogi with Korean Food mukbang 韓食焼肉 บุลโกกิอาหารเกาหลี,아미아미,구독자 4.21만명,조회수 3.4만회,1주 전,2019/11/17 22:39:20', '', '생크림 가~득 들어있는 쿠키슈 먹방(Dessert mukbang) fresh cream bread 生クリームパンขนมปังครีมสด,아미아미,구독자 4.21만명,조회수 4.3만회,1주 전,2019/11/17 22:39:20', '', '풀버전) 반올림피자샵 수제고구마피자 + 베이컨쉬림프피자 먹방 mukbang pizza,아미아미,구독자 4.21만명,조회수 1.9만회,1주 전,2019/11/17 22:39:20', '', '돈스파이크 스테이크에 한우 부위별로 먹방 !!! mukbang eating show,아미아미,구독자 4.21만명,조회수 2.3만회,1주 전,2019/11/17 22:39:20', '', '신상 라이트불닭볶음면7봉지+수육1kg 먹방 Spicy R

In [9]:
#/제거
import re
a=[]

for i in armyList:
    text=re.compile('[^ ㄱ-ㅣ가-힣]+') 
    result = text.sub(' ', i)
    a.append(result)
print(a)
print(type(a))

['뜨 끈하고 든든한 국밥 한 그릇 먹고 가세요                  아미아미 구독자  만명 조회수  만회 일 전   ', '', '해물짬뽕 한 그릇 탕수육 중식먹방                   아미아미 구독자  만명 조회수  만회 일 전   ', '', '김밥천국에서 소소하게 먹방      아미아미 구독자  만명 조회수  만회 일 전   ', '', '라면  봉지 먹었습니다             아미아미 구독자  만명 조회수  만회 일 전   ', '', '당면 가 득 넣고 뚝배기 불고기 만들어 먹방               아미아미 구독자  만명 조회수  만회 주 전   ', '', '생크림 가 득 들어있는 쿠키슈 먹방           아미아미 구독자  만명 조회수  만회 주 전   ', '', '풀버전  반올림피자샵 수제고구마피자   베이컨쉬림프피자 먹방    아미아미 구독자  만명 조회수  만회 주 전   ', '', '돈스파이크 스테이크에 한우 부위별로 먹방        아미아미 구독자  만명 조회수  만회 주 전   ', '', '신상 라이트불닭볶음면 봉지 수육  먹방            아미아미 구독자  만명 조회수  만회 주 전   ', '', '수제김밥 질리도록 먹방                  아미아미 구독자  만명 조회수  만회 주 전   ', '', '그린하우스  가지 빵 먹방        아미아미 구독자  만명 조회수  만회 주 전   ', '', ' 티코 구구아이스크림 왕창 먹방                아미아미 구독자  만명 조회수  만회 주 전   ', '', '찬바람불면 생각나는 우동  개만 먹을게요               아미아미 구독자  만명 조회수  만회 주 전   ', '', '불닭볶음면을 뿌링핫도그랑 같이 먹으면 맛있을까  먹방              아미아미 구독자  만명 조회수  만회 주 전   ', '', '스팸   김치찜에 계란후라이   건강에 해로운 영상                아미아미 구

In [10]:
armyRdd_=spark.sparkContext.parallelize(a)  

In [16]:
#불용어제거하고 빈도수 추출
stopwords = [u'전',u'넣은',u'천회',u'조회수',u'개월',u'년',u'봉지',u'먹을게요',u'가지',u'먹었습니다',u'같이',u'종',u'일',u'ㅈ봉지',u'리얼사운드먹방',u'개',u'아미아미',u'천회년',u'만회년',u'먹방아마따구독자',u'만회년',u'먹방',u'만명조회수',u'리얼사운드',u'구독자',u'만회일',u'먹고',u'만회개월',u'직접만든',u'주',u'만명',u'만회',u'개만',u'푸파',u'종류별로',u'마리',u'가',u'득',u'에',u'인분',u'한',u'그릇',u'그린하우스',u'입니다',u'간단하게',u'진짜',u'밥도둑',u'볶음밥까지']
armystop=armyRdd_.flatMap(lambda x:x.split()).filter(lambda x:x not in stopwords)
print(type(armystop))

<class 'pyspark.rdd.PipelinedRDD'>


In [17]:
armystop=armystop.collect()
print(armystop)
print(type(armystop))

['뜨', '끈하고', '든든한', '국밥', '가세요', '해물짬뽕', '탕수육', '중식먹방', '김밥천국에서', '소소하게', '라면', '당면', '넣고', '뚝배기', '불고기', '만들어', '생크림', '들어있는', '쿠키슈', '풀버전', '반올림피자샵', '수제고구마피자', '베이컨쉬림프피자', '돈스파이크', '스테이크에', '한우', '부위별로', '신상', '라이트불닭볶음면', '수육', '수제김밥', '질리도록', '빵', '티코', '구구아이스크림', '왕창', '찬바람불면', '생각나는', '우동', '불닭볶음면을', '뿌링핫도그랑', '먹으면', '맛있을까', '스팸', '김치찜에', '계란후라이', '건강에', '해로운', '영상', '까르보나라치킨', '크런치반반치킨', '요거닭', '단호박치즈', '오꾸닭', '등뼈찜에', '참치주먹밥', '당면이', '계속', '나와요', '유바유바', '모음집', '처음부터', '끝까지', '명랑핫도그', '전메뉴에', '신전떡볶이', '최고의', '조합입니다', '마리만', '자메이카', '시크릿반반', '떡만두국', '김치', '볶음편', '생생우동', '처음먹었는데', '맛있어요', '칠리새우', '라면에', '참치김밥은', '너무', '맛있어요ㅠ', 'ㅠ', '보들보들한족발이랑', '매운막국수', '에피타이저', '케이크', '판', '쿠키영상있어요', '진짜장', '봉지에', '계란후라이', '퓨전짬뽕', '뽕아저씨', '메뉴', '먹고왔습니다', '연어', '국수처럼', '이슬톡톡', '요즘', '핫한', '마피아떡볶이', '저도', '먹어봤습니다', '통삼겹바베큐에', '불닭쫄볶이', '간짬뽕먹방', '반찬이', '가지나', '나오는', '곱창집이라구요', '전투먹방', '자메이카에', '미역국', '새로운', '꿀조합', '탄생', '이삭토스트', '먹어보겠습니다', '노브랜드', '치즈치킨까스', '쫄면', '라면', '간단히', '먹으려다가', '상금', '만원', '떡볶

In [18]:
armystopRdd=spark.sparkContext.parallelize(armystop) 

In [23]:
#정규화한다음 빈도수 추출
army=armystopRdd\
    .flatMap(lambda x:x.split(','))\
    .map(lambda x:(x,1))\
    .reduceByKey(lambda x,y:x+y)\
    .map(lambda x:(x[1],x[0]))\
    .sortByKey(False)\
    .take(18)

In [24]:
print(army)
print(type(army))

[(6, '치즈볼'), (5, '라면'), (4, '스팸'), (4, '계란후라이'), (4, '김치'), (4, '연어'), (4, '떡볶이'), (3, '수육'), (3, '삼겹살'), (3, '김'), (3, '신제품'), (3, '까르보불닭'), (2, '불고기'), (2, '반올림피자샵'), (2, '한우'), (2, '빵'), (2, '명랑핫도그'), (2, '칠리새우')]
<class 'list'>


In [26]:
f=open("C:\\Users\\jmlee\\Code\\Frequency.txt",'a',-1,"utf-8")
A=len(army)
print (A)
for i in range (0,A):
    f.writelines(str(army[i]))
    f.writelines("\n")
f.close()

18
